*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [26]:
# import libraries (you may add additional imports but you may not have to)
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import requests
import zipfile
import numpy as np


In [27]:
# Downloading dataset
r = requests.get('https://cdn.freecodecamp.org/project-data/books/book-crossings.zip') 
open('book-crossings.zip', 'wb').write(r.content)

with zipfile.ZipFile('book-crossings.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [28]:
# Storing dataset into Pandas Dataframes
books = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']


# Removing users with less than 200 ratings and books with less than 100 ratings
counts1 = ratings['userID'].value_counts()
ratings = ratings[ratings['userID'].isin(counts1[counts1 >= 200].index)]
counts2 = ratings['bookRating'].value_counts()
ratings = ratings[ratings['bookRating'].isin(counts2[counts2 >= 100].index)]

/Users/jeremyg/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 24

In [29]:
# Combining ratings and books and removing unnecessary columns
combine_book_rating = pd.merge(ratings, books, on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL']
combine_book_rating = combine_book_rating.drop(columns, axis=1)

In [30]:
# Remove rows with no title
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

In [31]:
# Adding the total number of ratings and grouping per book
book_ratingCount = (combine_book_rating.
groupby(by = ['bookTitle'])['bookRating'].
count().
reset_index().
rename(columns = {'bookRating': 'totalRatingCount'})
[['bookTitle', 'totalRatingCount']]
)

In [32]:
# Merging the previous dataframe with the ratings+books dataframe
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'bookTitle', right_on = 'bookTitle', how = 'left')

In [57]:
# Removing duclicate ratings
rating_with_totalRatingCount = rating_with_totalRatingCount.drop_duplicates(['userID', 'bookTitle'])

# Reshaping rating_with_totalRatingCount to have book titles as indices, user IDs as columns and rating as values
rating_with_totalRatingCount_pivot_with_na = rating_with_totalRatingCount.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating')
rating_with_totalRatingCount_pivot = rating_with_totalRatingCount_pivot_with_na.fillna(0)

# Converting to sparse matrix for performance considerations
rating_with_totalRatingCount_matrix = csr_matrix(rating_with_totalRatingCount_pivot.values)

In [ ]:
# Creating and fitting the model
model_knn = NearestNeighbors(n_neighbors = rating_with_totalRatingCount_pivot.shape[0], metric = 'cosine')
fitted_model = model_knn.fit(rating_with_totalRatingCount_matrix)

book = "Where the Heart Is (Oprah's Book Club (Paperback))"
distances, indices = fitted_model.kneighbors(rating_with_totalRatingCount_pivot.loc[book].values.reshape(1, -1))

for i in range(0, 6):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_with_totalRatingCount_pivot.index[indices.flatten()[i]]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_with_totalRatingCount_pivot.index[indices.flatten()[i]], distances.flatten()[i]))


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [209]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7230184], ["The Pilot's Wife : A Novel", 0.81926787], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.823043], ['Bel Canto: A Novel', 0.8247875]]]
You haven't passed yet. Keep trying!
